# Planning-Lab Lesson 3: Markov Decision Process

In the third session we will work on the Markov decision process (MDP)

## Lava environments
The environments used are LavaFloor (visible in the figure) and its variations.

![Lava](images/lava.png)

The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls of the previous environments, the floor is covered with lava, there is a black pit of death.

Moreover, the agent can't comfortably perform its actions that instead have a stochastic outcome (visible in the figure):

![Dynact](images/dynact.png)

The action dynamics is the following:
- $P(0.8)$ of moving **in the desired direction**
- $P(0.1)$ of moving in a direction 90° with respect to the desired direction

Finally, since the floor is covered in lava, the agent receives a negative reward for each of its steps!

- -0.04 for each lava cell (L)
- -5 for the black pit (P). End of episode
- +1 for the treasure (G). End of episode

In [ ]:
import os, sys, random
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
from timeit import default_timer as timer
from tqdm import tqdm as tqdm

: 

### Environment Properties 

In addition to the varables of the environments you have been using in the previous sessions, there are also a few more:

- $T$: matrix of the transition function $T(s, a, s') \rightarrow [0, 1]$
- $RS$: matrix of the reward function $R(s) \rightarrow \mathbb{R}$

The available actions are still Left, Right, Up, Down.

#### Code Hints:

In [ ]:
env = gym.make("LavaFloor-v0")

current_state = env.pos_to_state(0, 0)
next_state = env.pos_to_state(0, 1)
goal_state = env.pos_to_state(2, 3)

print("Number of actions: ", env.action_space.n)
print("Actions: ", env.actions)
print("Reward of starting state:", env.RS[current_state])
print("Reward of goal state:", env.RS[goal_state])
print("Probability from (0, 0) to (0, 1) with action right:", env.T[current_state, 1, next_state])
print("Probability from (0, 0) to (2, 3) with action right:", env.T[current_state, 1, goal_state])

: 

In [ ]:
env_name = "LavaFloor-v0"
env = gym.make(env_name)

c=0
state_right = env.pos_to_state(0, 1) #state to the tight of start state
for i in range(1,101):
    current_state = env.pos_to_state(0, 0)
    state = env.sample(current_state, 1) #trying to go right
    if (state==state_right): 
        c+=1 #counting how many times the agent reaches the state to the right
        
#computing percentage of time agent reached the state to the right going right, should be around 80%           
print("percentage of time agent reaches the state to the right: ", c/i*100) 

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])


: 

## Assignment 1: Value Iteration Algorithm

Your first assignment is to implement the Value Iteration algorithm on LavaFloor. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state.  You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*, *NiceLavaFloor* and *VeryBadLavaFloor*.

<img src="images/value-iteration.png" width="600">

The *value_iteration* function has to be implemented. Notice that the value iteration approach return a matrix with the value for eacht state, the function *values_to_policy* automatically convert this matrix in the policy.

In [ ]:
def value_iteration(environment, maxiters=300, discount=0.9, max_error=1e-3):
    """
    Performs the value iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: timeout for the iterations
        discount: gamma value, the discount factor for the Bellman equation
        max_error: the maximum error allowd in the utility of any state
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    
    U_1 = [0 for _ in range(environment.observation_space.n)] # vector of utilities for states S
    delta = 0 # maximum change in the utility o any state in an iteration
    U = U_1.copy()
    
    for _ in range(maxiters):
        U_1 = U.copy()
        delta = 0

        for s in range(environment.observation_space.n):
            action_values = []
            for a in range(environment.action_space.n):
                expected_value = 0
                for next_state in range(environment.observation_space.n):
                    prob = environment.T[s, a, next_state]
                    reward = environment.RS[next_state]
                    expected_value += prob * (reward + discount * U_1[next_state])
                action_values.append(expected_value)
                
            U[s] = max(action_values)
            delta = max(delta, abs(U[s] - U_1[s]))

        if delta < max_error:
            break

    return values_to_policy(np.asarray(U), env) # automatically convert the value matrix U to a policy

: 

**The following code executes Value Iteration and prints the resulting policy**

In [ ]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"
#env_name = "NiceLavaFloor-v0"
#env_name = "VeryBadLavaFloor-v0"



env = gym.make(env_name)
print("\nENV RENDER:")
env.render()

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])

t = timer()
policy = value_iteration(env)

print("\nEXECUTION TIME: \n{}".format(round(timer() - t, 4)))
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
results = CheckResult_L3(env_name, policy_render)
results.check_value_iteration()

: 

## Assignment 2: Policy Iteration Algorithm (<span style="color:red">*OPTIONAL*</span>)

Your <span style="color:red"> *optional*</span> assignment is to implement the Policy Iteration algorithm on LavaFloor. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state. You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*, *NiceLavaFloor* and *VeryBadLavaFloor*.

<img src="images/policy-iteration.png" width="600">

For the *policy evaluation step*, it is necessary to implement this function:

<img src="images/policy-evaluating.png" width="500">

**The following function has to be implemented:**

In [ ]:
def policy_iteration(environment, maxiters=150, discount=0.9, maxviter=10):
    """
    Performs the policy iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: timeout for the iterations
        discount: gamma value, the discount factor for the Bellman equation
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    
    policy = [0 for _ in range(environment.observation_space.n)] #initial policy
    U = [0 for _ in range(environment.observation_space.n)] #utility array

    # Step (1): Policy Evaluation
    
    def policy_evaluation(policy, U, environment, discount, maxviter):
        for _ in range(maxviter):
            U_1 = U.copy()
            for s in range(environment.observation_space.n):
                a = policy[s]
                U[s] = sum([environment.T[s, a, next_state] * (environment.RS[next_state] + discount * U_1[next_state])
                            for next_state in range(environment.observation_space.n)])
            if np.max(np.abs(np.asarray(U) - np.asarray(U_1))) < 1e-3:
                break
        return U
    
    # Step (2) Policy Improvement
    unchanged = True  
    for _ in range(maxiters):
        # Policy Evaluation
        U = policy_evaluation(policy, U, environment, discount, maxviter)
        
        # Policy Improvement
        unchanged = True
        for s in range(environment.observation_space.n):
            action_values = []
            for a in range(environment.action_space.n):
                expected_value = sum([environment.T[s, a, next_state] * (environment.RS[next_state] + discount * U[next_state])
                                      for next_state in range(environment.observation_space.n)])
                action_values.append(expected_value)
            
            best_action = np.argmax(action_values)
            if policy[s] != best_action:
                policy[s] = best_action
                unchanged = False
        
        if unchanged:
            break
    
    return np.asarray(policy)

: 

**The following code executes and Value Iteration and prints the resulting policy**

In [ ]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"
#env_name = "NiceLavaFloor-v0"
#env_name = "VeryBadLavaFloor-v0"

env = gym.make(env_name)
print("\nENV RENDER:")
env.render()

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])

t = timer()
policy = policy_iteration(env)

print("\nEXECUTION TIME: \n{}".format(round(timer() - t, 4)))
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
results = CheckResult_L3(env_name, policy_render)
results.check_policy_iteration()

: 

## Comparison

The following code performs a comparison between Value Iteration and Policy Iteration, by plotting the accumulated rewards of each episode with iterations in range $[1, 50]$ (might take a long time if not optimizied via numpy). You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*.

The function **run_episode(envirnonment, policy, max_iteration)** run an episode on the given environment using the input policy.

In [ ]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])



maxiters = 49

env = gym.make(env_name)

series = []  # Series of learning rates to plot
liters = np.arange(maxiters + 1)  # Learning iteration values
liters[0] = 1
elimit = 100  # Limit of steps per episode
rep = 10  # Number of repetitions per iteration value
virewards = np.zeros(len(liters))  # Rewards array
c = 0

t = timer()

# Value iteration
for i in tqdm(liters, desc="Value Iteration", leave=True):
    reprew = 0
    policy = value_iteration(env, maxiters=i)  # Compute policy
        
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    virewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": virewards, "ls": "-", "label": "Value Iteration"})


vmaxiters = 5  # Max number of iterations to perform while evaluating a policy
pirewards = np.zeros(len(liters))  # Rewards array
c = 0

# Policy iteration
for i in tqdm(liters, desc="Policy Iteration", leave=True):
    reprew = 0
    policy = policy_iteration(env, maxiters=i)  # Compute policy
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    pirewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": pirewards, "ls": "-", "label": "Policy Iteration"})

print("Execution time: {0}s".format(round(timer() - t, 4)))
np.set_printoptions(linewidth=10000)

plot(series, "Learning Rate", "Iterations", "Reward")

: 

Correct results for comparison can be found here below. Notice that since the executions are stochastic the charts could differ: the important thing is the global trend and the final convergence to an optimal solution.

**Standard Lava floor results comparison**
<img src="images/results-standard.png" width="600">

**Huge Lava floor results comparison** 
<img src="images/results-huge.png" width="600">